TEST 1 - OK!

In [ ]:
from openai import AzureOpenAI
import numpy as np
import fitz
import os
import tiktoken
import json
from typing import List, Dict

client = AzureOpenAI(

)

def contar_tokens(texto: str) -> int:
    """Cuenta el número de tokens en un texto."""
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(texto))

def dividir_texto_en_chunks(texto: str, max_tokens: int = 8000) -> List[str]:
    """Divide el texto en chunks más pequeños respetando el límite de tokens."""
    chunks = []
    palabras = texto.split()
    chunk_actual = []
    tokens_actuales = 0
    
    for palabra in palabras:
        tokens_palabra = contar_tokens(palabra + " ")
        if tokens_actuales + tokens_palabra > max_tokens:
            chunks.append(" ".join(chunk_actual))
            chunk_actual = [palabra]
            tokens_actuales = tokens_palabra
        else:
            chunk_actual.append(palabra)
            tokens_actuales += tokens_palabra
    
    if chunk_actual:
        chunks.append(" ".join(chunk_actual))
    
    return chunks

def extraer_texto_pdf(ruta_pdf: str) -> str:
    """Extrae el texto completo de un PDF."""
    texto = ""
    pdf = fitz.open(ruta_pdf)
    for pagina in pdf:
        texto += pagina.get_text("text") + "\n"
    pdf.close()
    return texto.strip()

def extraer_puntos_clave(texto: str) -> dict:
    """Extrae puntos clave del texto usando GPT."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Eres un asistente especializado en analizar documentos técnicos y licitaciones.
                Analiza detalladamente el texto y extrae TODOS los puntos clave relevantes.
                DEBES responder SOLO con un JSON válido que siga exactamente esta estructura:
                {
                    "requisitos_tecnicos": ["requisito1", "requisito2", ...],
                    "experiencia_requerida": ["experiencia1", "experiencia2", ...],
                    "plazos": ["plazo1", "plazo2", ...],
                    "presupuesto": ["dato1", "dato2", ...],
                    "otros_requisitos": ["otro1", "otro2", ...]
                }
                Asegúrate de incluir TODOS los detalles relevantes encontrados.
                Si no encuentras información para alguna categoría, deja la lista vacía."""
            }, {
                "role": "user",
                "content": texto[:4000]
            }]
        )
        
        respuesta_texto = response.choices[0].message.content.strip()
        
        try:
            inicio_json = respuesta_texto.find('{')
            fin_json = respuesta_texto.rfind('}') + 1
            if inicio_json >= 0 and fin_json > 0:
                json_texto = respuesta_texto[inicio_json:fin_json]
                return json.loads(json_texto)
        except:
            pass
        
        return {
            "requisitos_tecnicos": [],
            "experiencia_requerida": [],
            "plazos": [],
            "presupuesto": [],
            "otros_requisitos": []
        }
    
    except Exception as e:
        print(f"Error al extraer puntos clave: {str(e)}")
        return {
            "requisitos_tecnicos": [],
            "experiencia_requerida": [],
            "plazos": [],
            "presupuesto": [],
            "otros_requisitos": []
        }

def obtener_embedding_promedio(chunks: List[str]) -> np.ndarray:
    """Obtiene el embedding promedio de múltiples chunks de texto."""
    embeddings = []
    for chunk in chunks:
        try:
            response = client.embeddings.create(
                input=chunk,
                model="text-embedding-3-large"
            )
            embeddings.append(np.array(response.data[0].embedding))
        except Exception as e:
            print(f"Error al procesar chunk: {str(e)}")
            continue
    
    if not embeddings:
        raise ValueError("No se pudo obtener ningún embedding válido")
    
    return np.mean(embeddings, axis=0)

def calcular_similitud_coseno(vector1: np.ndarray, vector2: np.ndarray) -> float:
    """Calcula la similitud coseno entre dos vectores con umbrales ajustados."""
    similitud_base = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    umbral_no_relacionado = 0.3
    umbral_relacionado = 0.45
    
    if similitud_base < umbral_no_relacionado:
        return max(0, (similitud_base / umbral_no_relacionado) * 15)
    elif similitud_base < umbral_relacionado:
        return 15 + ((similitud_base - umbral_no_relacionado) / 
                    (umbral_relacionado - umbral_no_relacionado)) * 45
    else:
        factor = (similitud_base - umbral_relacionado) / (1 - umbral_relacionado)
        return 60 + (factor * 40)

def calcular_similitud_semantica(puntos_clave1: dict, puntos_clave2: dict) -> float:
    """Calcula la similitud semántica con reglas más estrictas de puntuación."""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Eres un sistema experto en comparación de documentos técnicos.
                Evalúa la similitud entre documentos técnicos dando especial importancia a:
                1. Coincidencia en requisitos técnicos y experiencia requerida
                2. Alineación en plazos y presupuestos
                3. Compatibilidad en otros requisitos
                
                Reglas de puntuación:
                1. 90-100%: Documentos técnicamente idénticos o muy similares
                2. 70-89%: Documentos con alta compatibilidad técnica
                3. 50-69%: Documentos relacionados con algunas diferencias
                4. 0-49%: Documentos con diferencias significativas
                
                Retorna únicamente un número."""
            }, {
                "role": "user",
                "content": f"Documento 1 (Licitación): {json.dumps(puntos_clave1, ensure_ascii=False)}\n\nDocumento 2 (Oferta): {json.dumps(puntos_clave2, ensure_ascii=False)}"
            }]
        )
        resultado = response.choices[0].message.content.strip()
        resultado_limpio = ''.join(filter(lambda x: x.isdigit() or x == '.', resultado))
        return float(resultado_limpio)
    
    except Exception as e:
        print(f"Error al calcular similitud semántica: {str(e)}")
        return 0.0

def procesar_documento_mejorado(ruta_archivo: str) -> tuple:
    """Procesa un documento y retorna tanto su embedding como sus puntos clave."""
    texto = extraer_texto_pdf(ruta_archivo)
    puntos_clave = extraer_puntos_clave(texto)
    chunks = dividir_texto_en_chunks(texto)
    embedding = obtener_embedding_promedio(chunks)
    return embedding, puntos_clave

def filtrar_documentos_invalidos(resultados: List[Dict]) -> List[Dict]:
    """Filtra los documentos malos (con baja similitud) de los buenos."""
    return [res for res in resultados if res['porcentaje'] >= 20]

def calcular_similitud_total(embedding1: np.ndarray, embedding2: np.ndarray, 
                           puntos_clave1: dict, puntos_clave2: dict) -> float:
    """Calcula la similitud total combinando embedding y análisis semántico."""
    similitud_coseno = calcular_similitud_coseno(embedding1, embedding2)
    similitud_semantica = calcular_similitud_semantica(puntos_clave1, puntos_clave2)
    
    return 0.2 * similitud_coseno + 0.8 * similitud_semantica

def procesar_multiples_pdfs(ruta_ofertas: str, ruta_aplicaciones: str) -> List[Dict]:
    """Procesa múltiples PDFs con análisis semántico mejorado."""
    if not os.path.exists(ruta_ofertas):
        raise ValueError(f"El directorio de ofertas no existe: {ruta_ofertas}")
    if not os.path.exists(ruta_aplicaciones):
        raise ValueError(f"El directorio de aplicaciones no existe: {ruta_aplicaciones}")
    
    ofertas = [f for f in os.listdir(ruta_ofertas) if f.endswith('.pdf')]
    aplicaciones = [f for f in os.listdir(ruta_aplicaciones) if f.endswith('.pdf')]
    
    if not ofertas:
        raise ValueError("No se encontraron archivos PDF en el directorio de ofertas")
    if not aplicaciones:
        raise ValueError("No se encontraron archivos PDF en el directorio de aplicaciones")
    
    resultados = []
    
    for oferta in ofertas:
        print(f"\nProcesando oferta: {oferta}")
        ruta_oferta = os.path.join(ruta_ofertas, oferta)
        try:
            embedding_oferta, puntos_clave_oferta = procesar_documento_mejorado(ruta_oferta)
            
            for aplicacion in aplicaciones:
                print(f"Comparando con aplicación: {aplicacion}")
                ruta_aplicacion = os.path.join(ruta_aplicaciones, aplicacion)
                embedding_aplicacion, puntos_clave_aplicacion = procesar_documento_mejorado(ruta_aplicacion)
                
                similitud = calcular_similitud_total(
                    embedding_oferta, embedding_aplicacion,
                    puntos_clave_oferta, puntos_clave_aplicacion
                )
                
                resultados.append({
                    'oferta': oferta,
                    'aplicacion': aplicacion,
                    'porcentaje': similitud,
                    'puntos_clave_oferta': puntos_clave_oferta,
                    'puntos_clave_aplicacion': puntos_clave_aplicacion
                })
                
                print(f"Porcentaje de coincidencia: {similitud:.2f}%")
                print("Puntos clave encontrados en la oferta:")
                for categoria, puntos in puntos_clave_oferta.items():
                    if puntos:
                        print(f"\n{categoria.upper()}:")
                        for punto in puntos:
                            print(f"- {punto}")
                print("\nPuntos clave encontrados en la aplicación:")
                for categoria, puntos in puntos_clave_aplicacion.items():
                    if puntos:
                        print(f"\n{categoria.upper()}:")
                        for punto in puntos:
                            print(f"- {punto}")
                print("-" * 50)
                
        except Exception as e:
            print(f"Error procesando {oferta}: {str(e)}")
            continue
    
    return resultados

def main():
    ruta_ofertas = "C:\\Users\\aymen\\Desktop\\testofertas\\oferta"
    ruta_aplicaciones = "C:\\Users\\aymen\\Desktop\\testofertas"
    
    try:
        resultados = procesar_multiples_pdfs(ruta_ofertas, ruta_aplicaciones)
        resultados_filtrados = filtrar_documentos_invalidos(resultados)
        
        if resultados_filtrados:
            print("\n=== MEJORES COINCIDENCIAS ===")
            resultados_ordenados = sorted(resultados_filtrados, key=lambda x: x['porcentaje'], reverse=True)
            for i, r in enumerate(resultados_ordenados[:3], 1):
                print(f"\n{i}. Lugar:")
                print(f"   Oferta: {r['oferta']}")
                print(f"   Aplicación: {r['aplicacion']}")
                print(f"   Coincidencia: {r['porcentaje']:.2f}%")
                print("-" * 50)
        else:
            print("No se encontraron coincidencias relevantes.")

    except Exception as e:
        print(f"Error en la ejecución del programa: {str(e)}")

if __name__ == "__main__":
    main()


Procesando oferta: licitacion.pdf
Comparando con aplicación: oferta.pdf
Porcentaje de coincidencia: 31.44%
Puntos clave encontrados en la oferta:

PLAZOS:
- Duración sin prórrogas del contrato: 3 años.
- Prórrogas: 1 año.
- Fecha prevista de inicio del contrato: junio de 2022

PRESUPUESTO:
- Presupuesto base de licitación (IVA excluido): 1.800.000€
- Presupuesto base de licitación (IVA incluido): 2.178.000€
- Valor estimado del contrato (IVA excluido): 2.400.000€

OTROS_REQUISITOS:
- Número de expediente: MAD-2022-04-07-DESARROLLO-TAREAS-ASOCIADAS-A-RPAS
- Órgano de contratación: Comité de Compras Central
- Naturaleza: Servicio
- Modalidad: Acuerdo Marco
- N.º de Acuerdos Marco que se adjudicarán: 1
- División de la contratación en lotes: No
- CPV: 73330000 SERVICIOS DE DESARROLLO DE SOFTWARE PERSONALIZADO
- Forma de adjudicación: El contrato se adjudicará a la oferta con mejor relación calidad-precio
- Tramitación: Ordinaria
- Procedimiento: ABIERTO
- Plazo de confirmación de la part

TEST 2 - OK!

In [ ]:
from openai import AzureOpenAI
import numpy as np
import fitz
import os
import tiktoken
import json
from typing import List, Dict

client = AzureOpenAI(

)

def contar_tokens(texto: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(texto))

def dividir_texto_en_chunks(texto: str, max_tokens: int = 8000) -> List[str]:
    chunks = []
    palabras = texto.split()
    chunk_actual = []
    tokens_actuales = 0
    
    for palabra in palabras:
        tokens_palabra = contar_tokens(palabra + " ")
        if tokens_actuales + tokens_palabra > max_tokens:
            chunks.append(" ".join(chunk_actual))
            chunk_actual = [palabra]
            tokens_actuales = tokens_palabra
        else:
            chunk_actual.append(palabra)
            tokens_actuales += tokens_palabra
    
    if chunk_actual:
        chunks.append(" ".join(chunk_actual))
    
    return chunks

def extraer_texto_pdf(ruta_pdf: str) -> str:
    texto = ""
    pdf = fitz.open(ruta_pdf)
    for pagina in pdf:
        texto += pagina.get_text("text") + "\n"
    pdf.close()
    return texto.strip()

def extraer_puntos_clave(texto: str) -> dict:
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Analiza el documento y extrae la información en formato JSON.
                
                REGLAS IMPORTANTES:
                1. El sector debe ser específico (tecnologia, alimentacion, construccion, etc.)
                2. El tema_principal debe ser una descripción concisa pero informativa
                3. Extrae todos los requisitos importantes
                4. Si no estás seguro de algún campo, intenta inferir basándote en el contexto
                5. NUNCA uses 'desconocido' como valor
                
                ESTRUCTURA JSON:
                {
                    "tipo": "licitacion|oferta",
                    "sector": "sector_especifico",
                    "tema_principal": "descripción_concisa",
                    "requisitos": ["req1", "req2"],
                    "plazos": ["plazo1", "plazo2"],
                    "costes": ["coste1", "coste2"]
                }
                
                IMPORTANTE: 
                - Responde SOLO con el JSON
                - Sin texto adicional
                - Sin explicaciones
                - El JSON debe ser válido"""
            }, {
                "role": "user",
                "content": "Analiza este documento y extrae los puntos clave siguiendo el formato especificado: " + texto[:4000]
            }]
        )
        
        respuesta_texto = response.choices[0].message.content.strip()
        try:
            return json.loads(respuesta_texto)
        except:
            # Intenta limpiar la respuesta
            inicio = respuesta_texto.find('{')
            fin = respuesta_texto.rfind('}') + 1
            if inicio >= 0 and fin > 0:
                json_texto = respuesta_texto[inicio:fin]
                return json.loads(json_texto)
            raise
    except Exception as e:
        print(f"Error en extracción: {str(e)}")
        return {
            "tipo": "no_determinado",
            "sector": "general",
            "tema_principal": "documento general",
            "requisitos": [],
            "plazos": [],
            "costes": []
        }

def obtener_embedding_promedio(chunks: List[str]) -> np.ndarray:
    embeddings = []
    for chunk in chunks:
        try:
            response = client.embeddings.create(
                input=chunk,
                model="text-embedding-3-large"
            )
            embeddings.append(np.array(response.data[0].embedding))
        except Exception as e:
            print(f"Error en embedding: {str(e)}")
            continue
    
    if not embeddings:
        raise ValueError("No se pudo obtener embedding válido")
    
    return np.mean(embeddings, axis=0)

def calcular_similitud_coseno(vector1: np.ndarray, vector2: np.ndarray) -> float:
    similitud_base = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    umbral_no_relacionado = 0.3
    umbral_relacionado = 0.45
    
    if similitud_base < umbral_no_relacionado:
        return max(0, (similitud_base / umbral_no_relacionado) * 15)
    elif similitud_base < umbral_relacionado:
        return 15 + ((similitud_base - umbral_no_relacionado) / 
                    (umbral_relacionado - umbral_no_relacionado)) * 45
    else:
        factor = (similitud_base - umbral_relacionado) / (1 - umbral_relacionado)
        return 60 + (factor * 40)

def calcular_similitud_semantica(puntos_clave1: dict, puntos_clave2: dict) -> float:
    try:
        # Si los sectores son completamente diferentes, retornar valor bajo
        if (puntos_clave1['sector'] != 'general' and 
            puntos_clave2['sector'] != 'general' and 
            puntos_clave1['sector'] != puntos_clave2['sector']):
            return 10.0
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Evalúa la similitud entre una licitación y una oferta.
                
                CRITERIOS DE EVALUACIÓN:
                1. Coincidencia de sector y tema (40%)
                2. Cumplimiento de requisitos (30%)
                3. Alineación de plazos (15%)
                4. Aspectos económicos (15%)
                
                ESCALA:
                0-20% - Muy baja relación
                21-50% - Relación parcial
                51-80% - Buena relación
                81-100% - Excelente relación
                
                RETORNA SOLO UN NÚMERO (0-100)"""
            }, {
                "role": "user",
                "content": f"LICITACIÓN:\n{json.dumps(puntos_clave1, ensure_ascii=False)}\n\nOFERTA:\n{json.dumps(puntos_clave2, ensure_ascii=False)}"
            }]
        )
        resultado = response.choices[0].message.content.strip()
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', resultado)))
    except Exception as e:
        print(f"Error en similitud: {str(e)}")
        return 0.0

def analizar_ventajas_desventajas(puntos_clave_licitacion: dict, puntos_clave_oferta: dict) -> tuple:
    ventajas = []
    desventajas = []
    
    # Verificar compatibilidad de sector
    if (puntos_clave_licitacion['sector'] != 'general' and 
        puntos_clave_oferta['sector'] != 'general' and 
        puntos_clave_licitacion['sector'] != puntos_clave_oferta['sector']):
        return [], [
            f"✗ DESCALIFICADA: Sectores incompatibles",
            f"  Licitación: {puntos_clave_licitacion['sector']}",
            f"  Oferta: {puntos_clave_oferta['sector']}"
        ]
    
    # Verificar relación temática
    palabras_licitacion = set(puntos_clave_licitacion['tema_principal'].lower().split())
    palabras_oferta = set(puntos_clave_oferta['tema_principal'].lower().split())
    palabras_comunes = {'el', 'la', 'los', 'las', 'de', 'del', 'y', 'en', 'para', 'con', 'por'}
    
    palabras_significativas = palabras_licitacion - palabras_comunes
    palabras_coincidentes = set(palabra for palabra in palabras_significativas 
                               if any(palabra in oferta_palabra 
                                     for oferta_palabra in palabras_oferta))
    
    if not palabras_coincidentes and puntos_clave_licitacion['tema_principal'] != "documento general":
        return [], [
            f"✗ DESCALIFICADA: Temas no relacionados",
            f"  Licitación: {puntos_clave_licitacion['tema_principal']}",
            f"  Oferta: {puntos_clave_oferta['tema_principal']}"
        ]
    
    # Análisis de requisitos
    for req in puntos_clave_licitacion.get('requisitos', []):
        if any(req.lower() in app_req.lower() for app_req in puntos_clave_oferta.get('requisitos', [])):
            ventajas.append(f"✓ Cumple requisito: {req}")
        else:
            desventajas.append(f"✗ No cumple requisito: {req}")
    
    # Análisis de plazos
    if puntos_clave_oferta.get('plazos'):
        ventajas.append("✓ Define plazos")
        for plazo in puntos_clave_oferta['plazos'][:2]:
            ventajas.append(f"  → {plazo}")
    else:
        desventajas.append("✗ No especifica plazos")
    
    # Análisis de costes
    if puntos_clave_oferta.get('costes'):
        ventajas.append("✓ Incluye costes")
        for coste in puntos_clave_oferta['costes'][:2]:
            ventajas.append(f"  → {coste}")
    else:
        desventajas.append("✗ No incluye costes")
    
    return ventajas, desventajas

def calcular_similitud_total(embedding1: np.ndarray, embedding2: np.ndarray, 
                           puntos_clave1: dict, puntos_clave2: dict) -> float:
    # Verificar compatibilidad básica
    if (puntos_clave1['sector'] != 'general' and 
        puntos_clave2['sector'] != 'general' and 
        puntos_clave1['sector'] != puntos_clave2['sector']):
        return 0.0
    
    similitud_coseno = calcular_similitud_coseno(embedding1, embedding2)
    similitud_semantica = calcular_similitud_semantica(puntos_clave1, puntos_clave2)
    
    # Dar más peso a la similitud semántica
    return 0.3 * similitud_coseno + 0.7 * similitud_semantica

def procesar_documento_mejorado(ruta_archivo: str) -> tuple:
    texto = extraer_texto_pdf(ruta_archivo)
    puntos_clave = extraer_puntos_clave(texto)
    chunks = dividir_texto_en_chunks(texto)
    embedding = obtener_embedding_promedio(chunks)
    return embedding, puntos_clave

def procesar_multiples_pdfs(ruta_ofertas: str, ruta_aplicaciones: str) -> List[Dict]:
    if not all(os.path.exists(ruta) for ruta in [ruta_ofertas, ruta_aplicaciones]):
        raise ValueError("Directorios no encontrados")
    
    ofertas = [f for f in os.listdir(ruta_ofertas) if f.endswith('.pdf')]
    aplicaciones = [f for f in os.listdir(ruta_aplicaciones) if f.endswith('.pdf')]
    
    if not ofertas or not aplicaciones:
        raise ValueError("No se encontraron PDFs")
    
    resultados = []
    
    for oferta in ofertas:
        print(f"\nAnalizando licitación: {oferta}")
        ruta_oferta = os.path.join(ruta_ofertas, oferta)
        try:
            embedding_oferta, puntos_clave_oferta = procesar_documento_mejorado(ruta_oferta)
            
            for aplicacion in aplicaciones:
                print(f"\nEvaluando oferta: {aplicacion}")
                ruta_aplicacion = os.path.join(ruta_aplicaciones, aplicacion)
                embedding_aplicacion, puntos_clave_aplicacion = procesar_documento_mejorado(ruta_aplicacion)
                
                similitud = calcular_similitud_total(
                    embedding_oferta, embedding_aplicacion,
                    puntos_clave_oferta, puntos_clave_aplicacion
                )
                
                ventajas, desventajas = analizar_ventajas_desventajas(
                    puntos_clave_oferta, puntos_clave_aplicacion
                )
                
                resultados.append({
                    'oferta': oferta,
                    'aplicacion': aplicacion,
                    'porcentaje': similitud,
                    'ventajas': ventajas,
                    'desventajas': desventajas,
                    'sector_licitacion': puntos_clave_oferta['sector'],
                    'sector_oferta': puntos_clave_aplicacion['sector']
                })
                
                print(f"\nPuntuación de compatibilidad: {similitud:.2f}%")
                
                if similitud < 20:
                    print("\n⚠️ OFERTA CON BAJA COMPATIBILIDAD")
                    print(f"Licitación: {puntos_clave_oferta['sector']} - {puntos_clave_oferta['tema_principal']}")
                    print(f"Oferta: {puntos_clave_aplicacion['sector']} - {puntos_clave_aplicacion['tema_principal']}")
                
                if ventajas:
                    print("\nPuntos fuertes:")
                    for v in ventajas:
                        print(v)
                if desventajas:
                    print("\nAspectos a mejorar:")
                    for d in desventajas:
                        print(d)
                
                print("-" * 50)
                
        except Exception as e:
            print(f"Error en el procesamiento: {str(e)}")
            continue
    
    return resultados

def main():
    ruta_ofertas = "C:\\Users\\aymen\\Desktop\\testofertas\\oferta"
    ruta_aplicaciones = "C:\\Users\\aymen\\Desktop\\testofertas"
    
    try:
        resultados = procesar_multiples_pdfs(ruta_ofertas, ruta_aplicaciones)
        resultados_filtrados = [r for r in resultados if r['porcentaje'] >= 20]
        
        if resultados_filtrados:
            print("\n=== RANKING DE OFERTAS ===")
            for i, r in enumerate(sorted(resultados_filtrados, 
                                      key=lambda x: x['porcentaje'], 
                                      reverse=True)[:3], 1):
                print(f"\n{i}. {r['aplicacion']} - {r['porcentaje']:.2f}%")
                if r['ventajas']:
                    print("\nPuntos destacados:")
                    for v in r['ventajas'][:3]:
                        print(v)
                if r['desventajas']:
                    print("\nPrincipales mejoras necesarias:")
                    for d in r['desventajas'][:3]:
                        print(d)
        else:
            print("\nNo se encontraron ofertas con compatibilidad suficiente.")

    except Exception as e:
        print(f"Error en la ejecución: {str(e)}")

if __name__ == "__main__":
    main()


Analizando licitación: licitacion.pdf

Evaluando oferta: oferta.pdf

Puntuación de compatibilidad: 58.17%

Puntos fuertes:
✓ Define plazos
  → Fecha de Presentación
✓ Incluye costes
  → Desglose de costes detallado
  → Valor medio ponderado ajustado al presupuesto base de licitación

Aspectos a mejorar:
✗ No cumple requisito: Razón social
✗ No cumple requisito: N.I.F.
✗ No cumple requisito: Nº de proveedor Navantia (si aplica)
✗ No cumple requisito: Datos de contacto: nombre, teléfono y correo electrónico
✗ No cumple requisito: Indicar si se trata de una PYME según la Recomendación 2003/361/CE
--------------------------------------------------

Evaluando oferta: ofertatecnica.pdf

Puntuación de compatibilidad: 44.54%

Puntos fuertes:
✓ Define plazos
  → Publicado en la Plataforma de Contratación del Sector Público el 10-05-2022 a las 09:46 horas

Aspectos a mejorar:
✗ No cumple requisito: Razón social
✗ No cumple requisito: N.I.F.
✗ No cumple requisito: Nº de proveedor Navantia (si ap

TEST 3 - OK!

In [ ]:
from openai import AzureOpenAI
import numpy as np
import fitz
import os
import tiktoken
import json
from typing import List, Dict
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from datetime import datetime
import tempfile
import shutil


client = AzureOpenAI(

)

def contar_tokens(texto: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(texto))

def dividir_texto_en_chunks(texto: str, max_tokens: int = 8000) -> List[str]:
    chunks = []
    palabras = texto.split()
    chunk_actual = []
    tokens_actuales = 0
    
    for palabra in palabras:
        tokens_palabra = contar_tokens(palabra + " ")
        if tokens_actuales + tokens_palabra > max_tokens:
            chunks.append(" ".join(chunk_actual))
            chunk_actual = [palabra]
            tokens_actuales = tokens_palabra
        else:
            chunk_actual.append(palabra)
            tokens_actuales += tokens_palabra
    
    if chunk_actual:
        chunks.append(" ".join(chunk_actual))
    
    return chunks

def extraer_texto_pdf(ruta_pdf: str) -> str:
    texto = ""
    pdf = fitz.open(ruta_pdf)
    for pagina in pdf:
        texto += pagina.get_text("text") + "\n"
    pdf.close()
    return texto.strip()

def extraer_puntos_clave(texto: str) -> dict:
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Analiza el documento y extrae la información en formato JSON.
                
                REGLAS IMPORTANTES:
                1. El sector debe ser específico (tecnologia, alimentacion, construccion, etc.)
                2. El tema_principal debe ser una descripción concisa pero informativa
                3. Extrae todos los requisitos importantes
                4. Si no estás seguro de algún campo, intenta inferir basándote en el contexto
                5. NUNCA uses 'desconocido' como valor
                
                ESTRUCTURA JSON:
                {
                    "tipo": "licitacion|oferta",
                    "sector": "sector_especifico",
                    "tema_principal": "descripción_concisa",
                    "requisitos": ["req1", "req2"],
                    "plazos": ["plazo1", "plazo2"],
                    "costes": ["coste1", "coste2"]
                }"""
            }, {
                "role": "user",
                "content": texto[:4000]
            }]
        )
        
        respuesta_texto = response.choices[0].message.content.strip()
        try:
            return json.loads(respuesta_texto)
        except:
            inicio = respuesta_texto.find('{')
            fin = respuesta_texto.rfind('}') + 1
            if inicio >= 0 and fin > 0:
                json_texto = respuesta_texto[inicio:fin]
                return json.loads(json_texto)
            raise
    except Exception as e:
        print(f"Error en extracción: {str(e)}")
        return {
            "tipo": "no_determinado",
            "sector": "general",
            "tema_principal": "documento general",
            "requisitos": [],
            "plazos": [],
            "costes": []
        }

def obtener_embedding_promedio(chunks: List[str]) -> np.ndarray:
    embeddings = []
    for chunk in chunks:
        try:
            response = client.embeddings.create(
                input=chunk,
                model="text-embedding-3-large"
            )
            embeddings.append(np.array(response.data[0].embedding))
        except Exception as e:
            print(f"Error en embedding: {str(e)}")
            continue
    
    if not embeddings:
        raise ValueError("No se pudo obtener embedding válido")
    
    return np.mean(embeddings, axis=0)

def calcular_similitud_coseno(vector1: np.ndarray, vector2: np.ndarray) -> float:
    similitud_base = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    umbral_no_relacionado = 0.3
    umbral_relacionado = 0.45
    
    if similitud_base < umbral_no_relacionado:
        return max(0, (similitud_base / umbral_no_relacionado) * 15)
    elif similitud_base < umbral_relacionado:
        return 15 + ((similitud_base - umbral_no_relacionado) / 
                    (umbral_relacionado - umbral_no_relacionado)) * 45
    else:
        factor = (similitud_base - umbral_relacionado) / (1 - umbral_relacionado)
        return 60 + (factor * 40)

def calcular_similitud_semantica(puntos_clave1: dict, puntos_clave2: dict) -> float:
    try:
        if (puntos_clave1['sector'] != 'general' and 
            puntos_clave2['sector'] != 'general' and 
            puntos_clave1['sector'] != puntos_clave2['sector']):
            return 10.0
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Evalúa la similitud entre una licitación y una oferta.
                
                CRITERIOS DE EVALUACIÓN:
                1. Coincidencia de sector y tema (40%)
                2. Cumplimiento de requisitos (30%)
                3. Alineación de plazos (15%)
                4. Aspectos económicos (15%)
                
                ESCALA:
                0-20% - Muy baja relación
                21-50% - Relación parcial
                51-80% - Buena relación
                81-100% - Excelente relación
                
                RETORNA SOLO UN NÚMERO (0-100)"""
            }, {
                "role": "user",
                "content": f"LICITACIÓN:\n{json.dumps(puntos_clave1, ensure_ascii=False)}\n\nOFERTA:\n{json.dumps(puntos_clave2, ensure_ascii=False)}"
            }]
        )
        resultado = response.choices[0].message.content.strip()
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', resultado)))
    except Exception as e:
        print(f"Error en similitud: {str(e)}")
        return 0.0

def analizar_ventajas_desventajas(puntos_clave_licitacion: dict, puntos_clave_oferta: dict) -> tuple:
    ventajas = []
    desventajas = []
    
    if (puntos_clave_licitacion['sector'] != 'general' and 
        puntos_clave_oferta['sector'] != 'general' and 
        puntos_clave_licitacion['sector'] != puntos_clave_oferta['sector']):
        return [], [
            f"✗ DESCALIFICADA: Sectores incompatibles",
            f"  Licitación: {puntos_clave_licitacion['sector']}",
            f"  Oferta: {puntos_clave_oferta['sector']}"
        ]
    
    palabras_licitacion = set(puntos_clave_licitacion['tema_principal'].lower().split())
    palabras_oferta = set(puntos_clave_oferta['tema_principal'].lower().split())
    palabras_comunes = {'el', 'la', 'los', 'las', 'de', 'del', 'y', 'en', 'para', 'con', 'por'}
    
    palabras_significativas = palabras_licitacion - palabras_comunes
    palabras_coincidentes = set(palabra for palabra in palabras_significativas 
                               if any(palabra in oferta_palabra 
                                     for oferta_palabra in palabras_oferta))
    
    if not palabras_coincidentes and puntos_clave_licitacion['tema_principal'] != "documento general":
        return [], [
            f"✗ DESCALIFICADA: Temas no relacionados",
            f"  Licitación: {puntos_clave_licitacion['tema_principal']}",
            f"  Oferta: {puntos_clave_oferta['tema_principal']}"
        ]
    
    for req in puntos_clave_licitacion.get('requisitos', []):
        if any(req.lower() in app_req.lower() for app_req in puntos_clave_oferta.get('requisitos', [])):
            ventajas.append(f"✓ Cumple requisito: {req}")
        else:
            desventajas.append(f"✗ No cumple requisito: {req}")
    
    if puntos_clave_oferta.get('plazos'):
        ventajas.append("✓ Define plazos")
        for plazo in puntos_clave_oferta['plazos'][:2]:
            ventajas.append(f"  → {plazo}")
    else:
        desventajas.append("✗ No especifica plazos")
    
    if puntos_clave_oferta.get('costes'):
        ventajas.append("✓ Incluye costes")
        for coste in puntos_clave_oferta['costes'][:2]:
            ventajas.append(f"  → {coste}")
    else:
        desventajas.append("✗ No incluye costes")
    
    return ventajas, desventajas

def calcular_similitud_total(embedding1: np.ndarray, embedding2: np.ndarray, 
                           puntos_clave1: dict, puntos_clave2: dict) -> float:
    if (puntos_clave1['sector'] != 'general' and 
        puntos_clave2['sector'] != 'general' and 
        puntos_clave1['sector'] != puntos_clave2['sector']):
        return 0.0
    
    similitud_coseno = calcular_similitud_coseno(embedding1, embedding2)
    similitud_semantica = calcular_similitud_semantica(puntos_clave1, puntos_clave2)
    
    return 0.3 * similitud_coseno + 0.7 * similitud_semantica

def procesar_documento_mejorado(ruta_archivo: str) -> tuple:
    texto = extraer_texto_pdf(ruta_archivo)
    puntos_clave = extraer_puntos_clave(texto)
    chunks = dividir_texto_en_chunks(texto)
    embedding = obtener_embedding_promedio(chunks)
    return embedding, puntos_clave

def crear_grafico_radar(puntos_clave: dict) -> go.Figure:
    categorias = ['Requisitos', 'Plazos', 'Costes']
    valores = [
        len(puntos_clave.get('requisitos', [])),
        len(puntos_clave.get('plazos', [])),
        len(puntos_clave.get('costes', []))
    ]
    
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=valores,
        theta=categorias,
        fill='toself',
        name=puntos_clave.get('tema_principal', 'Documento')
    ))
    
    fig.update_layout(
        polar=dict(radialaxis=dict(visible=True, range=[0, max(valores) + 1])),
        showlegend=True,
        title="Análisis de Componentes"
    )
    
    return fig

def crear_grafico_comparativo(resultados: list) -> go.Figure:
    df = pd.DataFrame([
        {'Oferta': r['aplicacion'], 'Puntuación': r['porcentaje']}
        for r in resultados
    ])
    
    fig = px.bar(df, x='Oferta', y='Puntuación',
                 title='Comparación de Ofertas',
                 labels={'Puntuación': 'Puntuación (%)'})
    
    fig.update_layout(
        xaxis_tickangle=-45,
        yaxis_range=[0, 100]
    )
    
    return fig

def analizar_documentos(archivo_licitacion, archivos_oferta, progress=gr.Progress()):
    try:
        progress(0, desc="Iniciando análisis...")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            # Guardar licitación
            ruta_licitacion = os.path.join(temp_dir, "licitacion.pdf")
            with open(ruta_licitacion, 'wb') as f:
                f.write(archivo_licitacion)
            
            progress(0.2, desc="Procesando licitación...")
            embedding_licitacion, puntos_clave_licitacion = procesar_documento_mejorado(ruta_licitacion)
            
            resultados = []
            total_ofertas = len(archivos_oferta)
            
            for idx, archivo_oferta in enumerate(archivos_oferta, 1):
                progress(0.2 + (0.6 * idx/total_ofertas), 
                        desc=f"Analizando oferta {idx}/{total_ofertas}...")
                
                ruta_oferta = os.path.join(temp_dir, f"oferta_{idx}.pdf")
                with open(ruta_oferta, 'wb') as f:
                    f.write(archivo_oferta)
                
                embedding_oferta, puntos_clave_oferta = procesar_documento_mejorado(ruta_oferta)
                
                similitud = calcular_similitud_total(
                    embedding_licitacion, embedding_oferta,
                    puntos_clave_licitacion, puntos_clave_oferta
                )
                
                ventajas, desventajas = analizar_ventajas_desventajas(
                    puntos_clave_licitacion, puntos_clave_oferta
                )
                
                nombre_archivo = getattr(archivo_oferta, 'name', f"Oferta {idx}")
                
                resultados.append({
                    'aplicacion': nombre_archivo,
                    'porcentaje': similitud,
                    'ventajas': ventajas,
                    'desventajas': desventajas,
                    'puntos_clave_oferta': puntos_clave_oferta
                })
        
        progress(0.9, desc="Generando visualizaciones...")
        
        grafico_radar = crear_grafico_radar(puntos_clave_licitacion)
        grafico_comparativo = crear_grafico_comparativo(resultados)
        
        html_resultados = ""
        for r in sorted(resultados, key=lambda x: x['porcentaje'], reverse=True):
            html_resultados += f"""
            <div style="border: 1px solid #ddd; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3>📄 {r['aplicacion']}</h3>
                <h4>Puntuación: {r['porcentaje']:.2f}%</h4>
                
                <div style="margin-top: 10px;">
                    <h4 style="color: #28a745;">✅ Puntos Fuertes:</h4>
                    <ul>
                        {''.join(f'<li>{v}</li>' for v in r['ventajas'])}
                    </ul>
                </div>
                
                <div style="margin-top: 10px;">
                    <h4 style="color: #dc3545;">❌ Aspectos a Mejorar:</h4>
                    <ul>
                        {''.join(f'<li>{d}</li>' for d in r['desventajas'])}
                    </ul>
                </div>
            </div>
            """
        
        progress(1.0, desc="¡Análisis completado!")
        return grafico_radar, grafico_comparativo, gr.HTML(html_resultados)
    
    except Exception as e:
        return None, None, gr.HTML(f"<div style='color: red;'>Error en el análisis: {str(e)}</div>")

def crear_interfaz():
    with gr.Blocks(theme=gr.themes.Soft()) as interfaz:
        gr.Markdown("""
        # 📊 Analizador Inteligente de Licitaciones
        
        Sistema avanzado para análisis y comparación de ofertas
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                archivo_licitacion = gr.File(
                    label="📄 Licitación Base",
                    file_types=[".pdf"],
                    type="binary"
                )
                archivos_oferta = gr.Files(
                    label="📑 Ofertas a Analizar",
                    file_types=[".pdf"],
                    type="binary"
                )
                boton_analizar = gr.Button(
                    "🔍 Iniciar Análisis", 
                    variant="primary"
                )
        
        with gr.Tabs():
            with gr.TabItem("📊 Visualizaciones"):
                with gr.Row():
                    grafico_radar = gr.Plot()
                    grafico_comparativo = gr.Plot()
            
            with gr.TabItem("📋 Resultados Detallados"):
                resultados_html = gr.HTML()
        
        boton_analizar.click(
            fn=analizar_documentos,
            inputs=[archivo_licitacion, archivos_oferta],
            outputs=[grafico_radar, grafico_comparativo, resultados_html]
        )
    
    return interfaz

if __name__ == "__main__":
    app = crear_interfaz()
    app.launch(share=True)

TEST 4 

In [ ]:
from openai import AzureOpenAI
import numpy as np
import fitz
import os
import tiktoken
import json
from typing import List, Dict
import gradio as gr
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from datetime import datetime
import tempfile
import shutil

client = AzureOpenAI(

)

def contar_tokens(texto: str) -> int:
    encoding = tiktoken.get_encoding("cl100k_base")
    return len(encoding.encode(texto))

def dividir_texto_en_chunks(texto: str, max_tokens: int = 8000) -> List[str]:
    chunks = []
    palabras = texto.split()
    chunk_actual = []
    tokens_actuales = 0
    
    for palabra in palabras:
        tokens_palabra = contar_tokens(palabra + " ")
        if tokens_actuales + tokens_palabra > max_tokens:
            chunks.append(" ".join(chunk_actual))
            chunk_actual = [palabra]
            tokens_actuales = tokens_palabra
        else:
            chunk_actual.append(palabra)
            tokens_actuales += tokens_palabra
    
    if chunk_actual:
        chunks.append(" ".join(chunk_actual))
    
    return chunks

def extraer_texto_pdf(ruta_pdf: str) -> str:
    texto = ""
    pdf = fitz.open(ruta_pdf)
    for pagina in pdf:
        texto += pagina.get_text("text") + "\n"
    pdf.close()
    return texto.strip()
def extraer_puntos_clave(texto: str) -> dict:
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Analiza el documento y extrae la información en formato JSON.
                
                REGLAS IMPORTANTES:
                1. El sector debe ser específico (tecnologia, alimentacion, construccion, etc.)
                2. El tema_principal debe ser una descripción concisa pero informativa
                3. Extrae todos los requisitos importantes
                4. Si no estás seguro de algún campo, intenta inferir basándote en el contexto
                5. NUNCA uses 'desconocido' como valor
                
                ESTRUCTURA JSON:
                {
                    "tipo": "licitacion|oferta",
                    "sector": "sector_especifico",
                    "tema_principal": "descripción_concisa",
                    "requisitos": ["req1", "req2"],
                    "plazos": ["plazo1", "plazo2"],
                    "costes": ["coste1", "coste2"]
                }"""
            }, {
                "role": "user",
                "content": texto[:4000]
            }]
        )
        
        respuesta_texto = response.choices[0].message.content.strip()
        try:
            return json.loads(respuesta_texto)
        except:
            inicio = respuesta_texto.find('{')
            fin = respuesta_texto.rfind('}') + 1
            if inicio >= 0 and fin > 0:
                json_texto = respuesta_texto[inicio:fin]
                return json.loads(json_texto)
            raise
    except Exception as e:
        print(f"Error en extracción: {str(e)}")
        return {
            "tipo": "no_determinado",
            "sector": "general",
            "tema_principal": "documento general",
            "requisitos": [],
            "plazos": [],
            "costes": []
        }

def obtener_embedding_promedio(chunks: List[str]) -> np.ndarray:
    embeddings = []
    for chunk in chunks:
        try:
            response = client.embeddings.create(
                input=chunk,
                model="text-embedding-3-large"
            )
            embeddings.append(np.array(response.data[0].embedding))
        except Exception as e:
            print(f"Error en embedding: {str(e)}")
            continue
    
    if not embeddings:
        raise ValueError("No se pudo obtener embedding válido")
    
    return np.mean(embeddings, axis=0)

def calcular_similitud_coseno(vector1: np.ndarray, vector2: np.ndarray) -> float:
    similitud_base = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2))
    
    umbral_no_relacionado = 0.3
    umbral_relacionado = 0.45
    
    if similitud_base < umbral_no_relacionado:
        return max(0, (similitud_base / umbral_no_relacionado) * 15)
    elif similitud_base < umbral_relacionado:
        return 15 + ((similitud_base - umbral_no_relacionado) / 
                    (umbral_relacionado - umbral_no_relacionado)) * 45
    else:
        factor = (similitud_base - umbral_relacionado) / (1 - umbral_relacionado)
        return 60 + (factor * 40)
    
def calcular_similitud_semantica(puntos_clave1: dict, puntos_clave2: dict) -> float:
    try:
        if (puntos_clave1['sector'] != 'general' and 
            puntos_clave2['sector'] != 'general' and 
            puntos_clave1['sector'] != puntos_clave2['sector']):
            return 10.0
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{
                "role": "system",
                "content": """Evalúa la similitud entre una licitación y una oferta.
                
                CRITERIOS DE EVALUACIÓN:
                1. Coincidencia de sector y tema (40%)
                2. Cumplimiento de requisitos (30%)
                3. Alineación de plazos (15%)
                4. Aspectos económicos (15%)
                
                ESCALA:
                0-20% - Muy baja relación
                21-50% - Relación parcial
                51-80% - Buena relación
                81-100% - Excelente relación
                
                RETORNA SOLO UN NÚMERO (0-100)"""
            }, {
                "role": "user",
                "content": f"LICITACIÓN:\n{json.dumps(puntos_clave1, ensure_ascii=False)}\n\nOFERTA:\n{json.dumps(puntos_clave2, ensure_ascii=False)}"
            }]
        )
        resultado = response.choices[0].message.content.strip()
        return float(''.join(filter(lambda x: x.isdigit() or x == '.', resultado)))
    except Exception as e:
        print(f"Error en similitud: {str(e)}")
        return 0.0

def analizar_ventajas_desventajas(puntos_clave_licitacion: dict, puntos_clave_oferta: dict) -> tuple:
    ventajas = []
    desventajas = []
    
    if (puntos_clave_licitacion['sector'] != 'general' and 
        puntos_clave_oferta['sector'] != 'general' and 
        puntos_clave_licitacion['sector'] != puntos_clave_oferta['sector']):
        return [], [
            f"✗ DESCALIFICADA: Sectores incompatibles",
            f"  Licitación: {puntos_clave_licitacion['sector']}",
            f"  Oferta: {puntos_clave_oferta['sector']}"
        ]
    
    palabras_licitacion = set(puntos_clave_licitacion['tema_principal'].lower().split())
    palabras_oferta = set(puntos_clave_oferta['tema_principal'].lower().split())
    palabras_comunes = {'el', 'la', 'los', 'las', 'de', 'del', 'y', 'en', 'para', 'con', 'por'}
    
    palabras_significativas = palabras_licitacion - palabras_comunes
    palabras_coincidentes = set(palabra for palabra in palabras_significativas 
                               if any(palabra in oferta_palabra 
                                     for oferta_palabra in palabras_oferta))
    
    if not palabras_coincidentes and puntos_clave_licitacion['tema_principal'] != "documento general":
        return [], [
            f"✗ DESCALIFICADA: Temas no relacionados",
            f"  Licitación: {puntos_clave_licitacion['tema_principal']}",
            f"  Oferta: {puntos_clave_oferta['tema_principal']}"
        ]
    
    for req in puntos_clave_licitacion.get('requisitos', []):
        if any(req.lower() in app_req.lower() for app_req in puntos_clave_oferta.get('requisitos', [])):
            ventajas.append(f"✓ Cumple requisito: {req}")
        else:
            desventajas.append(f"✗ No cumple requisito: {req}")
    
    if puntos_clave_oferta.get('plazos'):
        ventajas.append("✓ Define plazos")
        for plazo in puntos_clave_oferta['plazos'][:2]:
            ventajas.append(f"  → {plazo}")
    else:
        desventajas.append("✗ No especifica plazos")
    
    if puntos_clave_oferta.get('costes'):
        ventajas.append("✓ Incluye costes")
        for coste in puntos_clave_oferta['costes'][:2]:
            ventajas.append(f"  → {coste}")
    else:
        desventajas.append("✗ No incluye costes")
    
    return ventajas, desventajas    
def calcular_similitud_total(embedding1: np.ndarray, embedding2: np.ndarray, 
                           puntos_clave1: dict, puntos_clave2: dict) -> float:
    if (puntos_clave1['sector'] != 'general' and 
        puntos_clave2['sector'] != 'general' and 
        puntos_clave1['sector'] != puntos_clave2['sector']):
        return 0.0
    
    similitud_coseno = calcular_similitud_coseno(embedding1, embedding2)
    similitud_semantica = calcular_similitud_semantica(puntos_clave1, puntos_clave2)
    
    return 0.3 * similitud_coseno + 0.7 * similitud_semantica

def procesar_documento_mejorado(ruta_archivo: str) -> tuple:
    texto = extraer_texto_pdf(ruta_archivo)
    puntos_clave = extraer_puntos_clave(texto)
    chunks = dividir_texto_en_chunks(texto)
    embedding = obtener_embedding_promedio(chunks)
    return embedding, puntos_clave

def crear_grafico_radar(puntos_clave: dict) -> go.Figure:
    categorias = ['Requisitos', 'Plazos', 'Costes']
    valores = [
        len(puntos_clave.get('requisitos', [])),
        len(puntos_clave.get('plazos', [])),
        len(puntos_clave.get('costes', []))
    ]
    
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=valores,
        theta=categorias,
        fill='toself',
        name=puntos_clave.get('tema_principal', 'Documento'),
        line=dict(color='#2E86C1', width=2),
        fillcolor='rgba(46, 134, 193, 0.3)'
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True, 
                range=[0, max(valores) + 1],
                gridcolor='#E8E8E8'
            ),
            angularaxis=dict(
                gridcolor='#E8E8E8'
            )
        ),
        showlegend=True,
        title=dict(
            text="Análisis de Componentes",
            font=dict(size=20)
        ),
        paper_bgcolor='white',
        plot_bgcolor='white'
    )
    
    return fig

def crear_grafico_comparativo(resultados: list) -> go.Figure:
    df = pd.DataFrame([
        {
            'Oferta': os.path.basename(r['aplicacion']),
            'Puntuación': r['porcentaje']
        }
        for r in resultados
    ])
    
    fig = px.bar(df, x='Oferta', y='Puntuación',
                 title='Comparación de Ofertas',
                 labels={'Puntuación': 'Puntuación (%)'},
                 color='Puntuación',
                 color_continuous_scale='viridis')
    
    fig.update_layout(
        xaxis_tickangle=-45,
        yaxis_range=[0, 100],
        title=dict(
            text='Comparación de Ofertas',
            font=dict(size=20)
        ),
        paper_bgcolor='white',
        plot_bgcolor='white',
        xaxis=dict(gridcolor='#E8E8E8'),
        yaxis=dict(gridcolor='#E8E8E8')
    )
    
    return fig
def analizar_documentos(archivo_licitacion, archivos_oferta, progress=gr.Progress()):
    try:
        progress(0, desc="Iniciando análisis...")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            # Guardar licitación
            ruta_licitacion = os.path.join(temp_dir, "licitacion.pdf")
            with open(ruta_licitacion, 'wb') as f:
                f.write(archivo_licitacion)
            
            progress(0.2, desc="Procesando licitación...")
            embedding_licitacion, puntos_clave_licitacion = procesar_documento_mejorado(ruta_licitacion)
            
            resultados = []
            total_ofertas = len(archivos_oferta)
            
            for idx, archivo_oferta in enumerate(archivos_oferta, 1):
                progress(0.2 + (0.6 * idx/total_ofertas), 
                        desc=f"Analizando oferta {idx}/{total_ofertas}...")
                
                # Generar un nombre único para cada oferta
                nombre_archivo = f"Oferta_{idx}.pdf"
                ruta_oferta = os.path.join(temp_dir, nombre_archivo)
                
                with open(ruta_oferta, 'wb') as f:
                    f.write(archivo_oferta)
                
                embedding_oferta, puntos_clave_oferta = procesar_documento_mejorado(ruta_oferta)
                
                similitud = calcular_similitud_total(
                    embedding_licitacion, embedding_oferta,
                    puntos_clave_licitacion, puntos_clave_oferta
                )
                
                ventajas, desventajas = analizar_ventajas_desventajas(
                    puntos_clave_licitacion, puntos_clave_oferta
                )
                
                resultados.append({
                    'aplicacion': nombre_archivo,
                    'porcentaje': similitud,
                    'ventajas': ventajas,
                    'desventajas': desventajas,
                    'puntos_clave_oferta': puntos_clave_oferta
                })
        progress(0.9, desc="Generando visualizaciones...")
        
        grafico_radar = crear_grafico_radar(puntos_clave_licitacion)
        grafico_comparativo = crear_grafico_comparativo(resultados)
        
        # Encontrar la mejor oferta
        mejor_oferta = max(resultados, key=lambda x: x['porcentaje'])
# En la función analizar_documentos, actualiza el html_mejor_oferta:
        html_mejor_oferta = f"""
        <div style="background: #1f2937; padding: 20px; border-radius: 15px; box-shadow: 0 4px 6px rgba(0, 0, 0, 0.2);">
            <div style="border-bottom: 2px solid #28a745; margin-bottom: 15px;">
                <h2 style="color: #28a745; margin: 0 0 10px 0;">🏆 Oferta Más Recomendada</h2>
            </div>
            
            <div style="background: #374151; padding: 15px; border-radius: 10px; margin-bottom: 15px;">
                <h3 style="color: #ffffff; margin: 0;">📄 {mejor_oferta['aplicacion']}</h3>
                <div style="background: #28a745; border-radius: 20px; padding: 5px 15px; display: inline-block; margin-top: 10px;">
                    <h4 style="color: white; margin: 0;">Puntuación: {mejor_oferta['porcentaje']:.2f}%</h4>
                </div>
            </div>
            
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; background: #1f2937;" >
                <div style="background: #374151; padding: 15px; border-radius: 10px; border-left: 4px solid #28a745;">
                    <h4 style="color: #28a745; margin-top: 0;">✅ Puntos Fuertes</h4>
                    <ul style="margin: 0; padding-left: 20px; color: #ffffff;">
                        {''.join(f'<li style="margin-bottom: 5px;">{v}</li>' for v in mejor_oferta['ventajas'][:3])}
                    </ul>
                </div>
                
                <div style="background: #374151; padding: 15px; border-radius: 10px; border-left: 4px solid #dc3545;">
                    <h4 style="color: #dc3545; margin-top: 0;">❌ Aspectos a Mejorar</h4>
                    <ul style="margin: 0; padding-left: 20px; color: #ffffff;">
                        {''.join(f'<li style="margin-bottom: 5px;">{d}</li>' for d in mejor_oferta['desventajas'][:3])}
                    </ul>
                </div>
            </div>
        </div>
        """

        html_resultados = ""
        for r in sorted(resultados, key=lambda x: x['porcentaje'], reverse=True):
            html_resultados += f"""
            <div style="border: 1px solid #ddd; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3>📄 {r['aplicacion']}</h3>
                <h4>Puntuación: {r['porcentaje']:.2f}%</h4>
                
                <div style="margin-top: 10px;">
                    <h4 style="color: #28a745;">✅ Puntos Fuertes:</h4>
                    <ul>
                        {''.join(f'<li>{v}</li>' for v in r['ventajas'])}
                    </ul>
                </div>
                
                <div style="margin-top: 10px;">
                    <h4 style="color: #dc3545;">❌ Aspectos a Mejorar:</h4>
                    <ul>
                        {''.join(f'<li>{d}</li>' for d in r['desventajas'])}
                    </ul>
                </div>
            </div>
            """
        
        progress(1.0, desc="¡Análisis completado!")
        return grafico_radar, grafico_comparativo, gr.HTML(html_resultados), gr.HTML(html_mejor_oferta)
    
    except Exception as e:
        return None, None, gr.HTML(f"<div style='color: red;'>Error en el análisis: {str(e)}</div>"), None

def crear_interfaz():
    with gr.Blocks(theme=gr.themes.Soft()) as interfaz:
        gr.Markdown("""
        # 📊 Analizador Inteligente de Licitaciones
        
        Sistema avanzado para análisis y comparación de ofertas
        """)
        
        with gr.Row():
            with gr.Column(scale=1):
                archivo_licitacion = gr.File(
                    label="📄 Licitación Base",
                    file_types=[".pdf"],
                    type="binary"
                )
                archivos_oferta = gr.Files(
                    label="📑 Ofertas a Analizar",
                    file_types=[".pdf"],
                    type="binary"
                )
                boton_analizar = gr.Button(
                    "🔍 Iniciar Análisis", 
                    variant="primary"
                )
        
        # Sección de mejor oferta
        mejor_oferta_html = gr.HTML()
        
        # Sección de gráficas en una fila separada
        with gr.Row():
            grafico_radar = gr.Plot()
            grafico_comparativo = gr.Plot()
        
        # Pestaña para resultados detallados
        with gr.Tabs():
            with gr.TabItem("📋 Resultados Detallados"):
                resultados_html = gr.HTML()
        
        boton_analizar.click(
            fn=analizar_documentos,
            inputs=[archivo_licitacion, archivos_oferta],
            outputs=[grafico_radar, grafico_comparativo, resultados_html, mejor_oferta_html]
        )
    
    return interfaz

if __name__ == "__main__":
    app = crear_interfaz()
    app.launch(share=True)